In [1]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# y -> outlet liquid temperature
# q(t) -> liquid flow rate
# Narx -> yhat(t+1|t) = f(y(t) ... y(t-3) u(t) ... u(t-9))

#read data
data = open('../exchanger/exchanger.dat')
lst = []
for line in data:
    lst += [line.split()]
time = [float(x[0]) for x in lst]
u = [float(x[1]) for x in lst]
y = [float(x[2]) for x in lst]

In [3]:
#divide identification set and validation set
ID_LENGHT = 3000
time_id = time[:ID_LENGHT]
u_id = u[:ID_LENGHT]
y_id = y[:ID_LENGHT]

time_val = time[ID_LENGHT:]
u_val = u[ID_LENGHT:]
y_val = y[ID_LENGHT:]

In [4]:
for k in range(100):
    mlp = MLPRegressor(hidden_layer_sizes=(6,7,7),activation="logistic", solver="sgd", max_iter=5000, \
                       verbose=True, random_state=k, batch_size=200)
    Y = np.array(y_id[10:len(y_id)])
    PHI = []
    for i in range(ID_LENGHT - 10):
        regressors = [u_id[i:i+10] + y_id[6+i:10+i]]
        PHI.append(regressors[0])
    PHI = np.array(PHI)
    model = mlp.fit(PHI,Y)
    print(k, model.loss_)

Iteration 1, loss = 3737.73331513
Iteration 2, loss = 641.49367048
Iteration 3, loss = 14.74722382
Iteration 4, loss = 24.14330551
Iteration 5, loss = 4.91774320
Iteration 6, loss = 1.66701729
Iteration 7, loss = 1.69361832
Iteration 8, loss = 1.56515723
Iteration 9, loss = 1.54886495
Iteration 10, loss = 1.54769247
Iteration 11, loss = 1.54720572
Iteration 12, loss = 1.54687345
Iteration 13, loss = 1.54705752
Iteration 14, loss = 1.54726156
Iteration 15, loss = 1.54713389
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
0 1.5471338934
Iteration 1, loss = 3596.58404060
Iteration 2, loss = 611.51904389
Iteration 3, loss = 14.11991358
Iteration 4, loss = 23.18807847
Iteration 5, loss = 4.78787487
Iteration 6, loss = 1.61676474
Iteration 7, loss = 1.68934348
Iteration 8, loss = 1.56570618
Iteration 9, loss = 1.54806100
Iteration 10, loss = 1.54774590
Iteration 11, loss = 1.54685357
Iteration 12, loss = 1.54713184
Iteration 13, loss = 1.54706826
It

C:\Users\Matteo\Anaconda3\envs\py35\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


47 1.54714539933
Iteration 1, loss = 3608.42507989
Iteration 2, loss = 562.87987686
Iteration 3, loss = 14.57135522
Iteration 4, loss = 23.52774615
Iteration 5, loss = 4.45424555
Iteration 6, loss = 1.64533962
Iteration 7, loss = 1.69384215
Iteration 8, loss = 1.55970535
Iteration 9, loss = 1.54711782
Iteration 10, loss = 1.54892797
Iteration 11, loss = 1.54724532
Iteration 12, loss = 1.54705385
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
48 1.54705384762
Iteration 1, loss = 3717.51337996
Iteration 2, loss = 671.65048834
Iteration 3, loss = 15.02835521
Iteration 4, loss = 23.84948022
Iteration 5, loss = 5.07019357
Iteration 6, loss = 1.64424550
Iteration 7, loss = 1.69799277
Iteration 8, loss = 1.56175133
Iteration 9, loss = 1.54697415
Iteration 10, loss = 1.54822276
Iteration 11, loss = 1.54783497
Iteration 12, loss = 1.54741116
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
49 1.54741116256
Ite

In [ ]:
 mlp = MLPRegressor(hidden_layer_sizes=(6,7,7),activation="logistic", solver="lbfgs", max_iter=5000, verbose=True, random_state=42)

In [ ]:
Y = np.array(y_id[10:len(y_id)])
PHI = []
for i in tqdm(range(ID_LENGHT - 10)):
    regressors = [u_id[i:i+10] + y_id[6+i:10+i]]
    PHI.append(regressors[0])
PHI = np.array(PHI)

In [ ]:
model = mlp.fit(PHI,Y)

In [ ]:
y_hat = model.predict(PHI)

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)

plt.subplot(211)
plt.plot(y_hat[:2990], color='blue')
plt.subplot(212)
plt.plot(y_id[10:3000], color='red')
plt.show()

In [ ]:
plt.plot(y_id[10:3000], color='red')
plt.plot(y_hat[:2990], color='blue')
plt.show()

In [ ]:
model.loss_

In [ ]:
model.n_iter_

In [ ]:
model.batch_size

In [ ]:
model.coefs_